In [1]:
import torch
import numpy as np
from  tqdm import tqdm


In [2]:
with open("../Data/01 - The Fellowship Of The Ring.txt","r",encoding='utf-8') as file:
    raw_txt1 = file.read()

with open("../Data/02 - The Two Towers.txt","r",encoding='utf-8') as file:
    raw_txt2 = file.read()

with open("../Data/03 - The Return Of The King.txt","r",encoding='utf-8') as file:
    raw_txt3 = file.read()

In [3]:
raw_txt = raw_txt1 + raw_txt2 + raw_txt3
# Not beautiful
stripped_txt = raw_txt.replace('\n\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ')


I chose to keep \n and punctuation to get a more fun output at the end, but it did reduce the accuracy slightly.

In [4]:
vocab = list(set(stripped_txt))
vocab.sort()
vocab_dict = {}
for i, character in enumerate(vocab):
    identity_vector = torch.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[character] = identity_vector

In [5]:
# L
L = 15
dataX = torch.zeros((len(stripped_txt) - L, L, len(vocab_dict)))
dataY = torch.zeros(len(stripped_txt) - L)
for i in range(len(stripped_txt) - L):
    x = torch.zeros((L, len(vocab_dict)))
    for j in range(L):
        x[j] = vocab_dict[stripped_txt[i+j]]
    dataX[i] = x
    dataY[i] = torch.argmax(vocab_dict[stripped_txt[i+L]])

In [6]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[torch.argmax(vocab_dict[key]).item()] = key
for x in dataX[0]:
    print(reversed_dict[torch.argmax(x).item()])
    print(x)
print(reversed_dict[dataY[0].item()])
print(dataY[0])

T
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
h
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
r
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [7]:
from torch.utils.data import Dataset


class MobyDickDataset(Dataset):

    def __init__(self, x, y):
        self.X = x
        self.Y = y

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return (x, y)

    def __len__(self):
        return self.X.shape[0]

In [9]:
dataset = MobyDickDataset(dataX, dataY)
print(len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [2400000, 148876])

2548876


In [10]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8192, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, shuffle=True, num_workers=0)

In [11]:
class MyLSTM(torch.nn.Module):


    def __init__(self, feature_size, hidden_size, nr_of_stacks, output_dim):
        super().__init__()
        self.LSTM = torch.nn.LSTM(feature_size, hidden_size, nr_of_stacks, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_dim)
        self.hidden_size = hidden_size

    def forward(self, x):
        out, (h, c) = self.LSTM(x)
        out = self.linear(h)
        out = out[0]
        return out

In [12]:
import datetime

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device {device}.")

def train(model, optimizer, loss_f, epochs, train_loader):
    model.train()
    for epoch in range(epochs):
        loss_train = 0.0
        train_bar = tqdm(total=len(train_loader), desc="Epoch progression")

        for xb, yb in train_loader:
            xb = xb.to(device=device)
            yb = yb.to(device=device)
            yb = yb.long()
            with torch.cuda.amp.autocast(enabled=True):
                out = model(xb)
                loss = loss_f(out, yb)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_train += loss.item()
                train_bar.update()

        print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))
        train_bar.close()

Device cuda.


In [13]:
model = MyLSTM(len(vocab_dict), len(vocab_dict), 1, len(vocab_dict))
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()
model.to(device=device)
loss.to(device=device)
train(model, opt, loss, 55, train_loader)

Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

2021-05-04 13:45:41.206200  |  Epoch 0  |  Training loss 3.011


Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

2021-05-04 13:46:14.252676  |  Epoch 1  |  Training loss 2.434


Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

...

2021-05-04 14:08:53.719976  |  Epoch 52  |  Training loss 1.374


Epoch progression:   0%|          | 1/293 [00:00<00:47,  6.21it/s]

2021-05-04 14:09:16.053824  |  Epoch 53  |  Training loss 1.372


Epoch progression: 100%|██████████| 293/293 [00:21<00:00, 13.33it/s]

2021-05-04 14:09:38.030943  |  Epoch 54  |  Training loss 1.370


In [14]:
def validate(model, val_loader, train_loader):
    """
    Plot training and validation accuracy
    """
    model.eval()
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

In [15]:
validate(model, val_loader, train_loader)

Accuracy train: 0.59
Accuracy val: 0.59


{'train': 0.59128625, 'val': 0.5869045380047825}

In [16]:
def generate_text(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(500):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        seed[0, -1] = vocab_dict[reversed_dict[torch.argmax(next_c).item()]]
        print(reversed_dict[torch.argmax(next_c).item()], end='')

seq_seed = "'Farewell, Arag"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]


for i, t in enumerate(seq_seed_tensor[0]):
    print(reversed_dict[torch.argmax(t).item()], end='')

'Farewell, Arag

In [17]:
generate_text(model, seq_seed_tensor)

'Farewell, Aragorn and the stream of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger of the stranger o

It repeats! This is because it always picks the most probable character which leads to a loop, so we use sampling to
get more diversity.

In [18]:
def generate_text_with_sampling(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(1000):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        test_c = np.random.choice(len(vocab_dict), 1, p=next_c.cpu().detach().numpy())
        seed[0, -1] = vocab_dict[reversed_dict[test_c[0]]]
        print(reversed_dict[test_c[0]], end='')

In [19]:
generate_text_with_sampling(model, seq_seed_tensor)

'Farewell, Aragorn.   Frodo sent you. Not in searly Faramir zabled and it could have far down them then they had not Sam closes, and not had been we won't dover rying all the ring swooding behind.'
   'It was rat made white was spoke and some door into time by the Barrow-daybe now Ouch starting throught them on for winiofed of me, he said have suselvested up in a heart of this nam of them. Sam looked himself mind where the mean of Gollum if rather, is than well, and his torestoner!' After it. Yet the Nine?'
   Then he should may had gone bight often this way the Orcs erranong her near stief south of the Draurfax.   They good the Gate and drew that Rayurasting doup, can it tollom Ents stirl of flowed and side' han on hwar, and the Fieds remaining that ley bettirst!'
   It can myselfary rodough, or to have been gond slowly things. The Many his heading and the altoged, and shice on his off Orcs, the openers seen. They' said Legotho's meen Shrowland they waters of Eorl sinding lasted pothe

In [20]:
train(model, opt, loss, 50, train_loader)

Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

2021-05-04 14:10:38.342663  |  Epoch 0  |  Training loss 1.368


Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

2021-05-04 14:11:08.762117  |  Epoch 1  |  Training loss 1.366


Epoch progression:   0%|          | 0/293 [00:00<?, ?it/s]

...

2021-05-04 14:30:12.729213  |  Epoch 47  |  Training loss 1.313


Epoch progression:   0%|          | 1/293 [00:00<00:46,  6.34it/s]

2021-05-04 14:30:34.317575  |  Epoch 48  |  Training loss 1.313


Epoch progression: 100%|██████████| 293/293 [00:21<00:00, 13.45it/s]

2021-05-04 14:30:56.098043  |  Epoch 49  |  Training loss 1.312


In [21]:
seq_seed = "'Do not be afra"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]

generate_text_with_sampling(model, seq_seed_tensor)

'Do not be afraid Gandalf had wit Sam light went their livin. His heavy deencefullow only in thir line, and called nowary felt bum he beyond-mitting to them beast, but is he need side now known you and there becaughared night some love and made no proving that he sprath,' said Pippin. `Come and master,' said Aragorn,' but the rider and becomed is. Unce you have like the mich of Bilbad Treebeard he must have you becile you on the Lord now had darce-foat companioned! You hasped, every kind graving but Boromir length in rike out. A patch of Cirith and Pernying, summoted of the road there few a news pilla.
   `So, well, without anxifuses, from the way in Years that have been polding it as then he capt to ode then and he stood by the save of the passion!'
   `Niding they foul there was friends of the linquentile south sair, efter of Moria curt. His thought there may was the new leave hideep rappond, howed
      Let behave his bruilly, dim into a man part side and they healds seemed in the g

In [22]:
validate(model, val_loader, train_loader)



Accuracy train: 0.61
Accuracy val: 0.60


{'train': 0.6062179166666667, 'val': 0.6008624627206535}